In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
print(openai.api_key )

### Select by similarity: 该对象根据与输入的相似性选择示例。它通过寻找与输入具有最大余弦相似度的嵌入的例子来做到这一点。

In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate



In [ ]:
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="Input:{input}\nOutput:{output}",
)

### 下面是examples,similarity selector会根据input，从中选择最相似的：

In [ ]:
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [ ]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of example available to select from.
    examples,
    # This is the embedding classused to produce embeddings which are used measure semantic similarity.
    OpenAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=1
)

similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:", 
    input_variables=["adjective"],
)

In [ ]:
# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(adjective="worried"))

In [ ]:
# Input is a measurement, so should select the tall/short example
print(similar_prompt.format(adjective="fat"))

In [ ]:
# You can add new examples to the SemanticSimilarityExampleSelector as well
similar_prompt.example_selector.add_example({"input": "enthusiastic", "output": "apathetic"})
print(similar_prompt.format(adjective="joyful"))